In [ ]:
from reframed import load_cbmodel
model = load_cbmodel("e_coli_core.xml.gz", flavor="bigg")

In [ ]:
from mewpy.optimization.evaluation import BPCY, TargetFlux
objs = [TargetFlux("R_EX_succ_e"), 
        BPCY(model.biomass_reaction, "R_EX_succ_e")]

In [ ]:
from mewpy.problems import RKOProblem
anaerobic = {'R_EX_o2_e': (0, 0)}
problem = RKOProblem(model, fevaluation=objs, envcond=anaerobic)

In [ ]:
from mewpy.optimization import EA
sols = EA(problem, max_generations=100).run()

In [ ]:
import pandas as pd
get_list = lambda x: [r_id[2:] for r_id in x.values]
table = [[get_list(x), len(get_list(x)), x.fitness[0], x.fitness[1]] for x in sols]
df = pd.DataFrame(table, columns=["reactions", "knockouts", "target", "BPCY"])           

In [ ]:
df.sort_values("knockouts")

In [ ]:
df.plot.scatter("BPCY", "target")